In [1]:
import pandas as pd

DATA_PATH = "LoLesports_data/"

train = pd.read_csv(f"{DATA_PATH}players_train.csv")
test = pd.read_csv(f"{DATA_PATH}players_test.csv")

train.shape, test.shape

((50000, 87), (11670, 87))

In [3]:
# 선수별 챔피언 통계 계산
player_champ_stats = (
    train.groupby(["playername", "champion"])
    .agg(
        {
            "result": ["count", "mean"],
            "kills": "mean",
            "deaths": "mean",
            "assists": "mean",
            "dpm": "mean",
            "damageshare": "mean",
            "vspm": "mean",
            "earnedgoldshare": "mean",
        }
    )
    .reset_index()
)

# 컬럼명 정리
player_champ_stats.columns = [
    "player",
    "champion",
    "games_played",
    "winrate",
    "avg_kills",
    "avg_deaths",
    "avg_assists",
    "avg_dpm",
    "avg_dmg_share",
    "avg_vspm",
    "avg_gold_share",
]

# 픽률 계산
total_games_by_player = train.groupby("playername")["result"].count().reset_index()
total_games_by_player.columns = ["player", "total_games"]

player_champ_stats = player_champ_stats.merge(total_games_by_player, on="player")
player_champ_stats["pickrate"] = (
    player_champ_stats["games_played"] / player_champ_stats["total_games"]
)

# 최소 5게임 이상 플레이한 데이터만 필터링
player_champ_stats = player_champ_stats[player_champ_stats["games_played"] >= 5]

# 승률과 픽률 기준으로 정렬
player_champ_stats.sort_values(["winrate", "pickrate"], ascending=False, inplace=True)

In [4]:
player_champ_stats["avg_kda"] = (
    player_champ_stats["avg_kills"] + player_champ_stats["avg_assists"]
) / player_champ_stats["avg_deaths"]

player_champ_stats.sort_values("games_played", ascending=False, inplace=True)
player_champ_stats

,player,champion,games_played,winrate,avg_kills,avg_deaths,avg_assists,avg_dpm,avg_dmg_share,avg_vspm,avg_gold_share,total_games,pickrate,avg_kda
2194,Delight,Rakan,59,0.644068,0.728814,1.881356,9.288136,129.493553,0.066892,3.566834,0.097626,261,0.226054,5.324324
3023,Faker,Azir,57,0.649123,2.684211,2.368421,4.842105,654.819895,0.293048,1.234049,0.233713,318,0.179245,3.177778
6553,Oner,Lee Sin,55,0.763636,3.927273,2.581818,7.236364,343.745176,0.151613,1.462816,0.190135,334,0.164671,4.323944
9781,Zeka,Azir,53,0.566038,3.226415,2.000000,4.094340,624.685438,0.297099,1.354540,0.259980,286,0.185315,3.660377
678,Bdd,Azir,50,0.600000,3.000000,1.880000,4.980000,575.999900,0.289891,1.295522,0.238223,255,0.196078,4.244681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7328,Ranger,Rell,5,0.400000,0.600000,3.800000,12.600000,129.267560,0.050315,3.428620,0.096753,84,0.059524,3.473684
8554,Tactical,Lucian,5,0.400000,2.400000,2.400000,3.000000,653.207360,0.295712,1.167260,0.256435,83,0.060241,2.250000
8552,Tactical,Kalista,5,0.400000,3.800000,2.800000,3.200000,374.829120,0.194780,1.234640,0.247289,83,0.060241,2.500000
2499,Dread,Vi,5,0.400000,3.200000,3.200000,8.200000,302.547080,0.120763,1.442980,0.172953,83,0.060241,3.562500


In [7]:
player_champ_stats.to_csv("output/players_champion_stat.csv", index=False)